# Coordle Backend

In [1]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain
import pymongo


import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import inspect
import sys

print(__name__, sys.argv[0])

__main__ /home/jonavin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py


In [3]:
df = pd.read_csv('data/cord-19-data.csv')

In [4]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

In [5]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [6]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index)

ai_index = AI_Index(model.wv.most_similar, n_similars=1)
ai_index.build_from_df(
    df[:16],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 16/16 [00:00<00:00, 269.04it/s]


In [7]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index, RecursiveDescentParser, AI_Index

docs, scores, errmsgs = ai_index.search('virus')

n = 69
if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')


dada
deque(['virus'])
1wswi7us  Relationship of SARS-CoV to other pathogenic RNA viruses explored by t  39.6302
yy96yeu9  Viral Discovery and Sequence Recovery Using DNA Microarrays             15.6958
xqhn0vbp  Airborne rhinovirus detection and effect of ultraviolet irradiation on  10.8963
qj4dh6rg  Cloaked similarity between HIV-1 and SARS-CoV suggests an anti-SARS st  8.0116
5s6acr7m  The Virus That Changed My World                                         7.2854
le0ogx1s  A new recruit for the army of the men of death                          7.0304
0qaoam29  A double epidemic model for the SARS propagation                        5.1278
gi6uaa83  Discovering human history from stomach bacteria                         2.6177
ng4rrdte  Pro/con clinical debate: Steroids are a key component in the treatment  1.9475
fy4w7xz8  Association of HLA class I with severe acute respiratory syndrome coro  0.9052
1769ovyk  8th Annual Toronto Critical Care Medicine Symposium, 30 October–1 Nove  0.4

In [8]:
ai_index.docmap['grace']

{1769ovyk}

In [9]:
with open('textfile.txt', 'w+') as f:
    uid = 'qzm9wgde'
    
    print(f'dumping {uid}')
    f.write(df[df.cord_uid == uid].body_text.values[0].lower())

dumping qzm9wgde


# Mongo

In [10]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index)

# ram_index = Index()
ram_index = AI_Index(model.wv.most_similar, 1)
ram_index.build_from_df(
    df[:128],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 128/128 [00:00<00:00, 392.29it/s]


In [11]:
model.wv.most_similar('corona')

[('coronavirus', 0.6643324494361877),
 ('coronovirus', 0.5752148032188416),
 ('coronaviruses', 0.5292266607284546),
 ('syncytial', 0.5095627307891846),
 ('rota', 0.5032534003257751),
 ('crownlike', 0.49367228150367737),
 ('paramyxo', 0.4860883057117462),
 ('flavi', 0.4837533235549927),
 ('herpes', 0.48291248083114624),
 ('picorna', 0.48181605339050293)]

In [20]:
import coordle_mongobackend as cm
reload(cm)

# mongoindex = cm.Index('coordle')
mongoindex = cm.AI_Index('coordle-lite', model.wv.most_similar, 1)
mongoindex.drop_old_collections()
mongoindex.build_from_df(df[:128], 'cord_uid', 'title', 'body_text', 
                         use_multiprocessing=True, workers=-1, verbose=True)

Text cleaning initilized on 16 workers


Initialized database index with size 0
Inserting wordmap to database: 100%|██████████| 21296/21296 [00:00<00:00, 264931.43it/s]


In [13]:
docs, scores, errmsgs = ram_index.search('virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}   {score:.4f}')

dada
deque(['virus'])
1wswi7us   6.9742
v95fzp8n   4.7759
rxbcuseq   3.9033
efrv5nvf   3.5226
jlhykfbf   3.1859
yy96yeu9   2.7622
azkamnpa   2.6476
1ke7i2wr   2.5464
0fitbwuv   2.4944
4k8f7ou1   2.3703
04cuk2cn   2.3648
jzj8q25c   2.3393
t20z4mtt   1.9912
dgnddq80   1.9810
eqiokrub   1.9740
xqhn0vbp   1.9176
pyam9yn3   1.8420
0brhn8oc   1.8389
4mnaicki   1.8203
fpj5urao   1.7773
jh9e85c0   1.6942
kqithgfo   1.6882
zowp10ts   1.6425
mxyxwkhx   1.6360
covwyddp   1.6148
zc491h8v   1.5761
1i36lsj2   1.5643
402ls2aq   1.4608
qj4dh6rg   1.4099
5s6acr7m   1.2821
4u2re1cu   1.2680
vblgotjn   1.2619
lwla5ugt   1.2553
le0ogx1s   1.2372
xtg0e142   1.1682
rrhh2alf   1.1382
e6e5nvn9   1.0813
qzm9wgde   1.0534
9k55f9o6   1.0082
5gsbtfag   0.9806
89xnnvuv   0.9740
ln8ddyuj   0.9677
0qaoam29   0.9024
p7um7o87   0.8935
m9rg6d3w   0.8529
qg0fsliy   0.7677
8zwsi4nk   0.7286
0gmtnkbh   0.7226
pqzwk6rj   0.6860
t40ybhgb   0.6422
0svscbpu   0.6012
mcfmxqp2   0.5998
ckby80vf   0.5846
ofx0hvvs   0.5652
2vlvz5

In [21]:
%%time
docs, scores, errmsgs = mongoindex.search('virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc}   {score:.4f}')

deque(['virus', 'OR', 'viruses'])
1wswi7us   6.9742
v95fzp8n   4.7759
rxbcuseq   3.9033
efrv5nvf   3.5226
jlhykfbf   3.1859
yy96yeu9   2.7622
azkamnpa   2.6476
1ke7i2wr   2.5464
0fitbwuv   2.4944
4k8f7ou1   2.3703
04cuk2cn   2.3648
jzj8q25c   2.3393
t20z4mtt   1.9912
dgnddq80   1.9810
eqiokrub   1.9740
xqhn0vbp   1.9176
pyam9yn3   1.8420
0brhn8oc   1.8389
4mnaicki   1.8203
fpj5urao   1.7773
jh9e85c0   1.6942
kqithgfo   1.6882
zowp10ts   1.6425
mxyxwkhx   1.6360
covwyddp   1.6148
zc491h8v   1.5761
1i36lsj2   1.5643
402ls2aq   1.4608
qj4dh6rg   1.4099
5s6acr7m   1.2821
4u2re1cu   1.2680
vblgotjn   1.2619
lwla5ugt   1.2553
le0ogx1s   1.2372
xtg0e142   1.1682
rrhh2alf   1.1382
e6e5nvn9   1.0813
qzm9wgde   1.0534
9k55f9o6   1.0082
5gsbtfag   0.9806
89xnnvuv   0.9740
ln8ddyuj   0.9677
0qaoam29   0.9024
p7um7o87   0.8935
m9rg6d3w   0.8529
qg0fsliy   0.7677
8zwsi4nk   0.7286
0gmtnkbh   0.7226
pqzwk6rj   0.6860
t40ybhgb   0.6422
0svscbpu   0.6012
mcfmxqp2   0.5998
ckby80vf   0.5846
ofx0hvvs   0

In [15]:
wordmap = {word:[doc.uid for doc in docs] for word, docs in ai_index.docmap.items() if len(word) < 100}

In [16]:
A = pd.value_counts(['a','a','a','d','c','c','c','c','d'], sort=False)
A.index.values

array(['d', 'c', 'a'], dtype=object)

In [17]:
A = [[],0]
A[0].append(1)
A[1] += 1
A

[[1], 1]

In [18]:
client = pymongo.MongoClient()
db_test = client['coordle']
db_test.wordcounts.count_documents({})

36565

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<center><h2> Junkyard </h2></center>

In [19]:
raise ValueError

ValueError: 

Creating sets from dicts is faster than from lists

In [ ]:
from time import time 
from matplotlib import pyplot as plt 
import numpy as np

dict_time = []
list_time = []
ratios = []

xrange = np.arange(1000,1000000,1000)
B = set(list(range(100000)))

for j in xrange:
    a = list(range(j))

    t0=time()
    A = set(a)
    A | B
    t1_list = time()-t0
    list_time.append(t1_list)
    
    a = {i:None for i in range(j)}
    
    t0=time()
#     A = set(a)
    a.keys() | B
    t1_dict = time()-t0
    dict_time.append(t1_dict)

    ratios.append(t1_list/t1_dict)

In [ ]:
plt.plot(xrange[1:], dict_time[1:], label='dict')
plt.plot(xrange[1:], list_time[1:], label='list')
plt.legend()
plt.show()

In [ ]:
plt.plot(xrange[1:], ratios[1:])
plt.show()

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2

queries = [
    'retarded!!!',
#     '(',
#     ')',
    'retarded (white AND (woman NOT man))',
    'retarded (white AND white) man',
#     'retarded OR white OR woman',
#     'retarded white AND woman',
#     'retarded OR white NOT woman',
#     'retarded (white NOT woman)',
#     'retarded (white NOT woman)',
#     'OR retarded AND white woman',
#     'retarded AND AND white NOT woman',
#     'retarded (white NOT woman) AND',
#     ')retarded ((white NOT woman) AND',
#     'retarded ((white NOT woman)',
#     'AND retarded)) ((white NOT woman) NOT',
]

rdp = RecursiveDescentParser(index.docmap)
for query in queries:
    errmsgs = []
    tokens = rdp.get_logical_querytokens(query)
    pass_ = rdp.assert_query(tokens, errmsgs)
    
#     print(tokens)
    print(rdp.parenthesis_handler(tokens))
#     print(errmsgs)
    print(pass_)
    print()
    